<a href="https://colab.research.google.com/github/codenoob1612/devrev_inter_iit/blob/main/Inter_IIT_taraksh_b21171.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 16.2 MB/s 
     |████████████████████████████████| 182 kB 39.9 MB/s 
     |████████████████████████████████| 7.6 MB 12.9 MB/s 


In [2]:
import torch
import numpy as np
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [3]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [120]:
def QnA(ques,passage,maxlen = 1000):
  input_ids = tokenizer_bert.encode(ques,passage,max_length = maxlen,truncation = True)
  sep_index = input_ids.index(102)
  len_ques = sep_index+1
  len_passage = len(input_ids) - len_ques
  segment_ids = [0]*len_ques + [1]*(len_passage)
  tokens = tokenizer_bert.convert_ids_to_tokens(input_ids)
  start_token_scores = model(torch.tensor([input_ids]),token_type_ids = torch.tensor([segment_ids]))[0]
  end_token_scores = model(torch.tensor([input_ids]),token_type_ids = torch.tensor([segment_ids]))[1]
  start_token_scores = start_token_scores.detach().numpy().flatten()
  end_token_scores = end_token_scores.detach().numpy().flatten()
  answer_start_index = np.argmax(start_token_scores)
  answer_end_index = np.argmax(end_token_scores)
  start_token_score = np.round(start_token_scores[answer_start_index], 2)
  end_token_score = np.round(end_token_scores[answer_end_index], 2)
  answer = tokens[answer_start_index]
  for i in range(answer_start_index + 1,answer_end_index + 1):
    if tokens[i][0:2] == '##':
      answer += tokens[i][2:]
    else:
      answer += ' ' + tokens[i]
  if (answer_start_index == 0) or (start_token_score<0) or (answer == "[SEP]") or (answer_end_index < answer_start_index):
    answer = ""
  return (answer_start_index,answer_end_index,answer)

In [6]:
import pandas as pd

In [7]:
paras = pd.read_csv(r"/content/drive/MyDrive/paragraphs.csv")
ques = pd.read_csv(r"/content/drive/MyDrive/questions.csv")

In [8]:
paras

,context
0,"In the 2000s, research in computer science, en..."
1,"Between 1847 and 1850, when Blore was building..."
2,One adaptation helping both predators and prey...
3,Schwarzenegger then went against the advice of...
4,Bell was a supporter of aerospace engineering ...
...,...
101,The Premier League is the most-watched sports ...
102,Joseph Robinette Biden Jr. or Joe Biden (born ...
103,The Shawshank Redemption is a 1994 American dr...
104,Dragon Ball Z is a Japanese anime television s...


In [9]:
ques

,question
0,Who was Hayek's father?
1,How does the lifecycle of most insects typical...
2,Who volunteer for the Tucson festival?
3,What practice did Altan Khan put to an end?
4,Who was the Needham Research Institute named a...
...,...
61,Who created the Dragon Ball manga series?
62,Who produced Dragon Ball Z?
63,What is the sequel to Dragon Ball Z?
64,Who is Son Goku's son?


In [18]:
ans = pd.read_csv(r"/content/drive/MyDrive/answers.csv")

In [17]:
ques["question"]

0                               Who was Hayek's father?
1     How does the lifecycle of most insects typical...
2                Who volunteer for the Tucson festival?
3           What practice did Altan Khan put to an end?
4     Who was the Needham Research Institute named a...
                            ...                        
61            Who created the Dragon Ball manga series?
62                          Who produced Dragon Ball Z?
63                 What is the sequel to Dragon Ball Z?
64                               Who is Son Goku's son?
65                    Who is the prince of the saiyans?
Name: question, Length: 66, dtype: object

In [21]:
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [24]:
!python -m spacy download en_core_web_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-17 13:37:51.698351: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 14 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [25]:
matcher = spacy.load("en_core_web_lg")

In [72]:
import nltk

In [76]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [84]:
def result(ques,para):
  words = nltk.word_tokenize(ques.lower())
  new_words= [word for word in words if word.isalnum()]
  Ques=""
  for word in new_words:
   if word not in set(stopwords.words("english")):
      Ques+=word + " "
  x = matcher(Ques)
  sim = 0
  ans = ""
  for i in para:
    Words = nltk.word_tokenize(i.lower())
    new_Words= [word for word in Words if word.isalnum()]
    Para=""
    for word in new_Words:
      if word not in set(stopwords.words("english")):
        Para+=word + " "
    y = matcher(Para)
    if x.similarity(y)>sim:
      sim = x.similarity(y)
      ans = i
  return ans

In [125]:
l = []
for i in ques["question"]:
  l.append(result(i,paras["context"]))

In [92]:
count=0
for i in range(47):
  if l[i]==z[i]:
    count+=1
count

30

In [124]:
ans_pos  = ans["answer_possible"]

In [118]:
text_ans = []
for i in range(66):
  text_ans.append(ans["answers"][i].split(":")[-1][2:-3])

In [131]:
task_1 = []
task_2 = []
for i in range(66):
  start,end,answer = QnA(ques["question"][i],l[i])
  task_2.append(answer)
  if ans_pos[i]==False and answer == "" :
    task_1.append("")
  else:
    task_1.append(l[i])

In [132]:
count_1=0
for i in range(66):
  if ans_pos[i]==False and task_1[i]=="":
    count_1+=1
  elif task_1[i]==ans["context"][i]:
    count_1+=1
count_1/66

0.696969696969697

In [155]:
import string

In [165]:
count_2=0
for i in range(66):
  x = text_ans[i].lower()
  if x !="" and task_2[i]!="":
    if x in task_2[i] or task_2[i] in x:
      count_2+=1
  elif x=="" and task_2[i]=="":
    count_2+=1
count_2/66

0.5454545454545454